In [7]:
#3 phase SFDI 
#imaging phantoms batch III with phantom batch II (A2) as reference
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.interpolate import griddata
import pandas as pd

%matplotlib qt

sf = ['0', '02'] #spatial frequencies  mm^{-1}
f = [0, 0.2] #spatial frequencies mm^{-1} 
fi = [0, 1, 2] #phases


Phantom = 'D3' #phantom we want to image - A3,B3,C3 or D3
refPhantom = 'A2' #A2 or A2ii

imagesref = [] #array to store the reference images 
for i in sf:
    for j in np.arange(0, len(fi), 1):
        #original
        im = (cv2.imread('Phantom{}_sf{}_phase{}_DCoffset_635_Warped.png'.format(refPhantom, i, j), 0)).astype(np.double)
        imagesref.append(im)

        
#Demodulation
def AC(var):
    return ((2)**(1/2)/3)*(((var[3]-var[4])**2 + (var[4]-var[5])**2 + (var[5]-var[3])**2)**(1/2))

def DC(var):
    return (1/3)*(var[0] + var[1] + var[2])


M_AC_ref = AC(imagesref) #AC modulation amplitude of reference phantom
M_DC_ref = DC(imagesref) #DC modulation ampltiude of reference phantom


images = [] #array to store phantom of interest data
for i in sf:
    for j in np.arange(0, len(fi), 1):
        im = (cv2.imread('Phantom{}_sf{}_phase{}_DCoffset_635_Warped.png'.format(Phantom, i, j), 0)).astype(np.double)
        images.append(im)
        
M_AC = AC(images) #AC modulation amplitude of phantom of interest
M_DC = DC(images) #DC modulation ampltiude of phantom of interest



####################REFERENCE PHANTOM OPTICAL PROPERTIES###########################
mu_a = 0 #Absorption coefficient mm^{-1}
mu_sp = 0.29 #Reduced scattering coefficient mm^{-1} 
n = 1.43 #refractive index  
###################################################################################


#Diffusion Approximation - calculating AC & DC Diffuse reflectance of reference pahtnom
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr

mu_effp_AC = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f[1])**2)/mu_tr**2)**(1/2)
Reflectance_AC = (3*A*ap)/(((mu_effp_AC/mu_tr)+1)*((mu_effp_AC/mu_tr)+3*A))

mu_effp_DC = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f[0])**2)/(mu_tr)**2)**(1/2)
Reflectance_DC = (3*A*ap)/(((mu_effp_DC/mu_tr)+1)*((mu_effp_DC/mu_tr)+3*A))


#Calculating AC & DC Diffuse reflectance of phantom of interest
#This piece takes any nan value and sets it to 0 so OP can be successfully calculated
R_d_AC = []
R_d_DC = []
for i in range(M_AC.shape[0]):
    for j in range(M_AC.shape[1]):
        if M_AC_ref[i][j] == 0:
            R_d_AC.append(0)
        else:
            R_d_AC.append((M_AC[i][j]/M_AC_ref[i][j])*Reflectance_AC)

for i in range(M_DC.shape[0]):
    for j in range(M_DC.shape[1]):
        if M_DC_ref[i][j] == 0:
            R_d_DC.append(0)
        else:
            R_d_DC.append((M_DC[i][j]/M_DC_ref[i][j])*Reflectance_DC) 


#reshaping diffuse reflectance images so they are original shape as input image
R_d_AC2 = np.reshape(R_d_AC, (M_AC.shape[0], M_AC.shape[1]))
R_d_DC2 = np.reshape(R_d_DC, (M_DC.shape[0], M_DC.shape[1]))


#plotting diffuse reflectance values of phantom of interest
plt.subplot(221), plt.imshow(R_d_AC2), plt.colorbar(), plt.title('R_AC'), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC2), plt.colorbar(), plt.title('R_DC'), plt.xticks([]), plt.yticks([])


#calculating optical properties
xi = []
#puts the DC and AC diffuse reflectance values into an array
for i in range(R_d_AC2.shape[0]): 
    for j in range(R_d_AC2.shape[1]):
        freq = [R_d_DC2[i][j], R_d_AC2[i][j]]
        xi.append(freq)


#Getting array of reflectance values and corresponding optical properties
mu_a_range = np.arange(0, 0.3, 0.001) #we are setting the absorption coefficient range
mu_sp_range = np.arange(0.1, 3, 0.01) #we are setting the reduces scattering coefficient range

#THE DIFFUSION APPROXIMATION
#getting the diffuse reflectance AC & DC values corresponding to specific absorption and reduced scattering coefficients
Reflectance_AC = []
Reflectance_DC = []
op_mua = []
op_sp = []
for i in range(len(mu_a_range)):
    for j in range(len(mu_sp_range)):
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a_range[i] + mu_sp_range[j]
        ap = mu_sp_range[j]/mu_tr
        
        mu_effp_AC = mu_tr*(3*(mu_a_range[i]/mu_tr) + ((2*np.pi*f[1])**2)/mu_tr**2)**(1/2)
        R_d1 = (3*A*ap)/(((mu_effp_AC/mu_tr)+1)*((mu_effp_AC/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)

        mu_effp_DC = mu_tr*(3*(mu_a_range[i]/mu_tr) + ((2*np.pi*f[0])**2)/mu_tr**2)**(1/2)
        R_d2 = (3*A*ap)/(((mu_effp_DC/mu_tr)+1)*((mu_effp_DC/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)

        op_mua.append(mu_a_range[i])

        op_sp.append(mu_sp_range[j])


#putting the DC and AC diffuse reflectance values generated from the Diffusion Approximation into an array    
points = []
for k in range(len(mu_a_range)*len(mu_sp_range)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

#putting the optical properties into two seperate arrays
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

#using scipy.interpolate.griddata to perform nearest interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_abs = griddata(points_array, op_mua_array, xi, method='nearest') #mua
coeff_sct = griddata(points_array, op_sp_array, xi, method='nearest') #musp

#reshaping to plot
abs_plot = np.reshape(coeff_abs, (R_d_AC2.shape[0], R_d_AC2.shape[1]))
sct_plot = np.reshape(coeff_sct, (R_d_AC2.shape[0], R_d_AC2.shape[1]))

#plotting optical property maps 
plt.subplot(223), plt.imshow(abs_plot), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([])
plt.colorbar()#, plt.clim(0.001, 0.3)


plt.subplot(224), plt.imshow(sct_plot), plt.title('Reduced scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()#, plt.clim(0.01, 1.5)


print("Absorption : ", np.mean(abs_plot))
print("Scattering: ", np.mean(sct_plot))


Absorption :  5.382918087372502e-05
Scattering:  0.5149113225105239


In [13]:
#3 phase SFDI 
#putting openSFDI data into code to check accuracy of code
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.interpolate import griddata
import pandas as pd

%matplotlib qt

sf = ['0', '02'] #spatial frequencies  mm^{-1}
f = [0, 0.2] #spatial frequencies mm^{-1} 
fi = [0, 1, 2] #phases


imagesref = [] #array to store the reference images 
for i in sf:
    for j in np.arange(0, len(fi), 1):
        #original
        #im = (cv2.imread('OpenSFDI_sf{}_phase{}_ref.tif'.format(i, j), 0)).astype(np.double)
        #cropped
        im = (cv2.imread('OpenSFDI_sf{}_phase{}_ref_CROP.png'.format(i, j), 0)).astype(np.double)
        imagesref.append(im)

        
#Demodulation
def AC(var):
    return ((2)**(1/2)/3)*(((var[3]-var[4])**2 + (var[4]-var[5])**2 + (var[5]-var[3])**2)**(1/2))

def DC(var):
    return (1/3)*(var[0] + var[1] + var[2])


M_AC_ref = AC(imagesref) #AC modulation amplitude of reference phantom
M_DC_ref = DC(imagesref) #DC modulation ampltiude of reference phantom


images = [] #array to store phantom of interest data
for i in sf:
    for j in np.arange(0, len(fi), 1):
        #original
        #im = (cv2.imread('OpenSFDI_sf{}_phase{}_hand.tif'.format(i, j), 0)).astype(np.double)
        #cropped
        im = (cv2.imread('OpenSFDI_sf{}_phase{}_hand_CROP.png'.format(i, j), 0)).astype(np.double)
        images.append(im)
        
M_AC = AC(images) #AC modulation amplitude of phantom of interest
M_DC = DC(images) #DC modulation ampltiude of phantom of interest



####################REFERENCE PHANTOM OPTICAL PROPERTIES###########################
mu_a = 0.0059 #Absorption coefficient mm^{-1}
mu_sp = 0.9748 #Reduced scattering coefficient mm^{-1} 
n = 1.43 #refractive index  
###################################################################################


#Diffusion Approximation - calculating AC & DC Diffuse reflectance of reference pahtnom
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr

mu_effp_AC = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f[1])**2)/mu_tr**2)**(1/2)
Reflectance_AC = (3*A*ap)/(((mu_effp_AC/mu_tr)+1)*((mu_effp_AC/mu_tr)+3*A))

mu_effp_DC = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f[0])**2)/(mu_tr)**2)**(1/2)
Reflectance_DC = (3*A*ap)/(((mu_effp_DC/mu_tr)+1)*((mu_effp_DC/mu_tr)+3*A))


#Calculating AC & DC Diffuse reflectance of phantom of interest
#This piece takes any nan value and sets it to 0 so OP can be successfully calculated
R_d_AC = []
R_d_DC = []
for i in range(M_AC.shape[0]):
    for j in range(M_AC.shape[1]):
        if M_AC_ref[i][j] == 0:
            R_d_AC.append(0)
        else:
            R_d_AC.append((M_AC[i][j]/M_AC_ref[i][j])*Reflectance_AC)

for i in range(M_DC.shape[0]):
    for j in range(M_DC.shape[1]):
        if M_DC_ref[i][j] == 0:
            R_d_DC.append(0)
        else:
            R_d_DC.append((M_DC[i][j]/M_DC_ref[i][j])*Reflectance_DC) 


#reshaping diffuse reflectance images so they are original shape as input image
R_d_AC2 = np.reshape(R_d_AC, (M_AC.shape[0], M_AC.shape[1]))
R_d_DC2 = np.reshape(R_d_DC, (M_DC.shape[0], M_DC.shape[1]))


#plotting diffuse reflectance values of phantom of interest
plt.subplot(221), plt.imshow(R_d_AC2), plt.colorbar(), plt.title('R_AC'), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC2), plt.colorbar(), plt.title('R_DC'), plt.xticks([]), plt.yticks([])


#calculating optical properties
xi = []
#puts the DC and AC diffuse reflectance values into an array
for i in range(R_d_AC2.shape[0]): 
    for j in range(R_d_AC2.shape[1]):
        freq = [R_d_DC2[i][j], R_d_AC2[i][j]]
        xi.append(freq)


#Getting array of reflectance values and corresponding optical properties
mu_a_range = np.arange(0, 0.3, 0.001) #we are setting the absorption coefficient range
mu_sp_range = np.arange(0.1, 3, 0.01) #we are setting the reduces scattering coefficient range

#THE DIFFUSION APPROXIMATION
#getting the diffuse reflectance AC & DC values corresponding to specific absorption and reduced scattering coefficients
Reflectance_AC = []
Reflectance_DC = []
op_mua = []
op_sp = []
for i in range(len(mu_a_range)):
    for j in range(len(mu_sp_range)):
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a_range[i] + mu_sp_range[j]
        ap = mu_sp_range[j]/mu_tr
        
        mu_effp_AC = mu_tr*(3*(mu_a_range[i]/mu_tr) + ((2*np.pi*f[1])**2)/mu_tr**2)**(1/2)
        R_d1 = (3*A*ap)/(((mu_effp_AC/mu_tr)+1)*((mu_effp_AC/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)

        mu_effp_DC = mu_tr*(3*(mu_a_range[i]/mu_tr) + ((2*np.pi*f[0])**2)/mu_tr**2)**(1/2)
        R_d2 = (3*A*ap)/(((mu_effp_DC/mu_tr)+1)*((mu_effp_DC/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)

        op_mua.append(mu_a_range[i])

        op_sp.append(mu_sp_range[j])


#putting the DC and AC diffuse reflectance values generated from the Diffusion Approximation into an array    
points = []
for k in range(len(mu_a_range)*len(mu_sp_range)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

#putting the optical properties into two seperate arrays
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

#using scipy.interpolate.griddata to perform nearest interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_abs = griddata(points_array, op_mua_array, xi, method='nearest') #mua
coeff_sct = griddata(points_array, op_sp_array, xi, method='nearest') #musp

#reshaping to plot
abs_plot = np.reshape(coeff_abs, (R_d_AC2.shape[0], R_d_AC2.shape[1]))
sct_plot = np.reshape(coeff_sct, (R_d_AC2.shape[0], R_d_AC2.shape[1]))

#plotting optical property maps 
plt.subplot(223), plt.imshow(abs_plot), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([])
plt.colorbar()#, plt.clim(0.001, 0.3)


plt.subplot(224), plt.imshow(sct_plot), plt.title('Reduced scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()#, plt.clim(0.01, 1.5)


print("Absorption : ", np.mean(abs_plot))
print("Scattering: ", np.mean(sct_plot))


Absorption :  0.005950530303030304
Scattering:  0.9725765151515148


In [ ]:
#Raspberry Pi code for image capture
import numpy as np
import picamera
import time
from fractions import Fraction

with picamera.PiCamera() as camera:
    camera.resolution = (1000,1000)
    camera.framerate = Fraction(1,6)#30
    #wait for the automatic gain control to settle
    camera.start_preview()
    time.sleep(2)
    #now fix the values
    camera.shutter_speed = 600000#camera.exposure_speed
    camera.exposure_mode = 'off'
    g = camera.awb_gains
    camera.awb_mode = 'off'
    camera.awb_gains = g
    camera.iso = 800
    camera.capture('PhantomA2_sf02_phase2_DCoffset_635.png')
